In [1]:
from os import listdir
from os.path import isfile, join
import pandas as pd 

data_path = '/Users/robert/git_RCA/drive-download-20211018T064401Z-001'
files = [f for f in listdir(data_path) if isfile(join(data_path, f))]
len(files)

54

# 一 : 從所有data中抓出 '國家名稱'&'產品代號'  

In [2]:
#set只會保留unique value 
#note: 原始檔中 其中一個UK的K是小寫 我就直接修改檔名了 

countries = set()
for file_name in files: 
    countries.add(file_name.split('_')[0]) #將國家名字拆出來
countries

{'Australia',
 'Brazil',
 'Canada',
 'China',
 'France',
 'Germany',
 'Italy',
 'Korea',
 'Malaysia',
 'Myanmar',
 'NewZealand',
 'Philippines',
 'Russian',
 'Singapore',
 'Taiwan',
 'UK',
 'US',
 'VietNam'}

In [3]:
print(f'總共國家數: {len(countries)}')

總共國家數: 18


# 二: 先測試單一國家的計算 以台灣的data為例  


In [4]:
#測試一下檔名&收尋方式 以taiwan為例
country_list = [] #>>>找出檔名中有taiwan的檔案 並將其存在country_list中
file_code = [] #>>>將檔名最後的編號取出來 方便最後儲存檔案用
for file_name in files:
    if 'Taiwan' in file_name:
        country_list.append(file_name)
        file_code.append(file_name.split('e')[-1])
print(country_list)
print(file_code)

['Taiwan_importvalue22060010006.xlsx', 'Taiwan_importvalue220890.xlsx', 'Taiwan_importvalue220830.xlsx']
['22060010006.xlsx', '220890.xlsx', '220830.xlsx']


時間的長度為2012/1~2020/12  
總共會有12*9 = 108個col

In [5]:
#不能用index_col = 0，因為資料columns的排序並不一致，有時country在前、有時 hscode在前
data_1 = pd.read_excel(data_path + '/' + country_list[0]).set_index('country')
data_2 = pd.read_excel(data_path + '/' + country_list[1]).set_index('country')
data_3 = pd.read_excel(data_path + '/' + country_list[2]).set_index('country')

In [19]:
data_3

,hscode,value201201,value201202,value201203,value201204,value201205,value201206,value201207,value201208,value201209,...,value202003,value202004,value202005,value202006,value202007,value202008,value202009,value202010,value202011,value202012
country,,,,,,,,,,,,,,,,,,,,,
Australia,22083000000,0,0,0,0,0,0,0,0,0,...,1,13,0,0,0,0,0,0,0,0
Austria,22083000000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,27,0,0,0
Belgium,22083000000,0,0,0,0,0,0,0,0,0,...,0,1,0,2,14,7,18,0,0,0
Bhutan,22083000000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Bulgaria,22083000000,0,0,0,0,0,0,0,0,9,...,0,0,0,13,0,0,0,0,0,0
Canada,22083000000,3,46,14,10,18,1,43,8,0,...,7,32,0,2,2,23,0,5,27,2
China,22083000000,0,0,0,0,0,0,0,0,0,...,0,0,0,3,0,0,0,0,0,0
Cyprus,22083000000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Czech Republic,22083000000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
class RCA_calulator:
    def __init__(self, data1, data2, data3, country_name):
        #確認column 總共有108個 若col數量和108對不上時，報錯
        assert len(data1.columns[1::]) == 108
        assert len(data2.columns[1::]) == 108
        assert len(data3.columns[1::]) == 108
        
        self.data1 = data1
        self.data2 = data2
        self.data3 = data3
        self.data1_hs = data1['hscode'][0]
        self.data2_hs = data2['hscode'][0]
        self.data3_hs = data3['hscode'][0]
        self.country_name = country_name
        
        '''分母的分母''' #sum>>每一個col垂直加總 最後再將三張表同月份的值加總
        self.total_Denominator = (self.data1.sum() + self.data2.sum() + self.data3.sum())[1::]
        
        '''最後的分母'''  
        self.child1 = self.data1.sum()[1::] / self.total_Denominator
        self.child2 = self.data2.sum()[1::] / self.total_Denominator
        self.child3 = self.data3.sum()[1::] / self.total_Denominator
        
        
        '''分子的分母'''  #總表 全部3張表直接水平加總 若缺失的部分會自動是補0
        self.all_molecular_of_Denominator = self.data1.add(self.data2.add(self.data3, fill_value=0), fill_value=0)
        
        '''最後的分子'''
        self.final_m_1 = (self.data1/self.all_molecular_of_Denominator.loc[self.data1.index]).fillna(0)
        self.final_m_2 = (self.data2/self.all_molecular_of_Denominator.loc[self.data2.index]).fillna(0)
        self.final_m_3 = (self.data3/self.all_molecular_of_Denominator.loc[self.data3.index]).fillna(0)
        
        '''RCA'''
        self.final_1 = (self.final_m_1 / self.child1).fillna(0)
        self.final_2 = (self.final_m_2 / self.child2).fillna(0)
        self.final_3 = (self.final_m_3 / self.child3).fillna(0)
        
        '''將hscode保留'''
        self.final_1.loc[:,'hscode'] = self.data1_hs
        self.final_2.loc[:,'hscode'] = self.data2_hs
        self.final_3.loc[:,'hscode'] = self.data3_hs
        
    def save(self):
        self.final_1.to_excel(f'/Users/robert/git_RCA/RCA_final/RCA_{self.country_name}{file_code[0]}')
        self.final_2.to_excel(f'/Users/robert/git_RCA/RCA_final/RCA_{self.country_name}{file_code[1]}')
        self.final_3.to_excel(f'/Users/robert/git_RCA/RCA_final/RCA_{self.country_name}{file_code[2]}')

In [8]:
data = RCA_calulator(data_1,data_2,data_3,'Taiwan')

# 檢視計算過程: 以taiwan第一個column 2012/01為例

In [26]:
'''分母的分母'''
print(data.total_Denominator)

value201201    27339
value201202    25073
value201203    24151
value201204    21265
value201205    18037
               ...  
value202008    39027
value202009    61710
value202010    57486
value202011    58713
value202012    61401
Length: 108, dtype: int64


27339為Taiwan 3張表2012/01值的總和

In [11]:
'''最後的分母'''  
#這邊的child1是對應data1的分母 
#以2012/01來說的話 是第一個col (504+6)/27339 = 0.0186
print(data.child1)

value201201    0.018655
value201202    0.029673
value201203    0.022442
value201204    0.015613
value201205    0.026778
                 ...   
value202008    0.020396
value202009    0.019770
value202010    0.017830
value202011    0.028716
value202012    0.038713
Length: 108, dtype: float64


In [12]:
'''分子的分母'''  #總表 全部3張表直接水平加總 若缺失的部分會自動是補0
print(data.all_molecular_of_Denominator)

                           hscode  value201201  value201202  value201203  \
country                                                                    
Australia            2.208300e+10          0.0          0.0          0.0   
Austria              2.208322e+10          0.0          0.0          0.0   
Belgium              2.208300e+10          0.0          0.0          0.0   
Bhutan               2.208300e+10          0.0          0.0          0.0   
Bulgaria             2.208300e+10          0.0          0.0          0.0   
Canada               2.208322e+10          3.0         46.0         14.0   
China                4.414323e+10          0.0         16.0          0.0   
Cyprus               2.208300e+10          0.0          0.0          0.0   
Czech Republic       2.208300e+10          0.0          0.0          0.0   
Denmark              2.208322e+10          0.0          0.0          0.0   
Finland              2.208300e+10          0.0          0.0          0.0   
France      

In [17]:
#我們來觀察一下‘Japan‘的值好了 
data.all_molecular_of_Denominator.loc['Japan','value201201']

1243.0

1243正好是三張表Japan對台灣出口的加總  
分別是 504 + 73 + 666 = 1243. 

In [28]:
'''最後的分子'''
#已data_1為例的話 就是504
#而最後的分子為 504/1243 = 0.4054 我們來檢查看看
data.final_m_1['value201201']

country
China            0.000000
France           0.000000
Japan            0.405471
Korea            0.130435
United States    0.000000
Name: value201201, dtype: float64

In [29]:
#最終data1的RCA Japan對Taiwan = 0.4054 / 0.0186 = 21.7956
#些微的運算差為浮點位數問題
data.final_1['value201201']

country
China             0.000000
France            0.000000
Japan            21.735611
Korea             6.992072
United States     0.000000
Name: value201201, dtype: float64

# 三:跑全部18個國家

In [11]:
#NewZealand_importdvalue220890 其column名稱hscode 多打了一個c 故程式會出錯 >> 已手動更正xlsx內容

for country_name in list(countries):
    #將某國的檔名&hscode提取
    country_list = [] 
    file_code = [] 
    for file_name in files:
        if country_name in file_name:
            country_list.append(file_name)
            file_code.append(file_name.split('e')[-1])
    #計算RCA並儲存
    data_1 = pd.read_excel(data_path + '/' + country_list[0]).set_index('country')
    data_2 = pd.read_excel(data_path + '/' + country_list[1]).set_index('country')
    data_3 = pd.read_excel(data_path + '/' + country_list[2]).set_index('country')
    data = RCA_calulator(data_1,data_2,data_3,country_name)
    data.save()

# OVER